# Letterments kata

The target of this kata is to convert a word to a list of possible strings with [periodic table element names](http://www.ptable.com/) from the original word's letters, for example:
```
letterments('cab') = ['Calcium Boron']
```
The result of the function is a list, since there are words that might have several solutions.
In case there is no possible result at all for a word, the function will return an empty list.

Solution to this kata should provide tests, further than the provided cases here.
In order to do that, there is a [pretty extensive list of words that might be translated to elemens](https://gist.github.com/jeffThompson/7789182), which would be used.

Let's see some examples:

In [1]:
cases = [
    ('c', ['Carbon']),
    ('z', []),
    ('al', ['Aluminum']),
    ('bo', ['Boron Oxygen']),
    ('ab', []),
    ('cab', ['Calcium Boron']),
    ('joy', []),
    ('bin', ['Boron Indium', 'Bismuth Nitrogen', 'Boron Iodine Nitrogen']),
]

def letterments_example(word):
    for case in cases:
        if case[0] == word:
            return case

In [2]:
for case in cases:
    print("letterments('{}') = {}".format(case[0], letterments_example(case[0])))

letterments('c') = ('c', ['Carbon'])
letterments('z') = ('z', [])
letterments('al') = ('al', ['Aluminium'])
letterments('ab') = ('ab', [])
letterments('cab') = ('cab', ['Calcium Boron'])
letterments('joy') = ('joy', [])
letterments('bin') = ('bin', ['Boron Indium', 'Bismuth Nitrogen', 'Boron Iodine Nitrogen'])


## Tooling for this dojo

In first place, this goodie to run our `unittest` test class has been included:

In [3]:
import sys
import unittest

def test(testClass):
    suite = unittest.TestLoader().loadTestsFromTestCase(testClass)
    unittest.TextTestRunner(verbosity=1, stream=sys.stderr).run(suite)

To ease to work with periodic table elements, usage of the `periodictable` module is recommended:

In [4]:
import periodictable

This way, a complete set of symbols and elements are already available...

... which can be queried by the `get_element_name` function.
This function gets a symbol, and returns the capitalized element name if symbol already exists, or `None`, otherwise.

**WARNING**: For some weird reason, `periodictable` includes Deuterium (D) as an element:

In [5]:
def get_element_name(symbol):
    try:
        return eval("periodictable.{}.name".format(symbol.capitalize())).capitalize()
    except AttributeError:
        pass

In [6]:
for case in cases:
    print("get_element_name('{}') = {}".format(case[0], get_element_name(case[0])))
print("get_element_name('d') = {}".format(get_element_name('d')))

get_element_name('c') = Carbon
get_element_name('z') = None
get_element_name('al') = Aluminum
get_element_name('ab') = None
get_element_name('cab') = None
get_element_name('joy') = None
get_element_name('bin') = None
get_element_name('d') = Deuterium


## Approaches

Some approaches to solve this kata could be already discarded, due to complexity of the problem, but some possible options were suggested:

* Brute force by iteration: Iterating over the letters in the word, to replace.
* Brute force by split: Creating a list of possible divisions of the word, like `[['w', 'o', 'r', 'd'], []`, and iterating over these, replacing each part by elements, and discarding the incomplete ones.
* Brute force by replacing: Using `str.replace` or regular expressions, to replace symbols by element names incrementally.
* Using concurrency to check every possibility.
* [Backtracking](https://en.wikipedia.org/wiki/Backtracking)

In [25]:
class LettermentsTest(unittest.TestCase):
    def test_single_letter(self):
        self.assertEqual(letterments('c'), ['Carbon'])
        self.assertEqual(letterments('z'), [])

    def test_double_letter(self):
        self.assertEqual(letterments('al'), ['Aluminum'])
        self.assertEqual(letterments('bo'), ['Boron Oxygen'])

In [79]:
def try_harder(word):
    elements = []
    if len(word) > 0:
        a, *b = word
        first = get_element_name(a)
        if first is not None:
            elements.append(first)
            elements.append(try_harder("".join(b)))
    if len(elements) > 0:
        return " ".join(elements).strip()
    else:
        return []

def letterments(word):
    solutions = []
    solution = get_element_name(word)
    if solution is not None:
        solutions.append(solution)
    else:
        solutions.append(try_harder(word))
    return solutions

In [80]:
test(LettermentsTest)

EF
ERROR: test_double_letter (__main__.LettermentsTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-25-9a49f0c691a1>", line 8, in test_double_letter
    self.assertEqual(letterments('bo'), ['Boron Oxygen'])
  File "<ipython-input-79-3e27781ea93c>", line 20, in letterments
    solutions.append(try_harder(word))
  File "<ipython-input-79-3e27781ea93c>", line 8, in try_harder
    elements.append(try_harder("".join(b)))
  File "<ipython-input-79-3e27781ea93c>", line 10, in try_harder
    return " ".join(elements).strip()
TypeError: sequence item 1: expected str instance, list found

FAIL: test_single_letter (__main__.LettermentsTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-25-9a49f0c691a1>", line 4, in test_single_letter
    self.assertEqual(letterments('z'), [])
AssertionError: Lists differ: [[]] != []

First list con